In [2]:
import pandas as pd
from SwingTrading import SwingTrading

In [4]:
#Create a SwingTrading object
swing_trading = SwingTrading('AAPL', '2020-01-01', '2020-12-31', '1d', 0.05, 0.03, 5)
# Retrieve the data
data = swing_trading.get_data()
data = swing_trading.forward_calc('High', 'max')

[*********************100%***********************]  1 of 1 completed


In [5]:
data.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,High_max
242,2020-12-16,127.410004,128.369995,126.559998,127.809998,126.293922,98208600,134.410004
243,2020-12-17,128.899994,129.580002,128.039993,128.699997,127.173378,94359800,134.410004
244,2020-12-18,128.960007,129.100006,126.120003,126.660004,125.157570,192541500,137.339996
245,2020-12-21,125.019997,128.309998,123.449997,128.229996,126.708946,121251600,138.789993
246,2020-12-22,131.610001,134.410004,129.649994,131.880005,130.315643,168904800,138.789993
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,129.406586,88223700,NaN
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,130.404587,54930100,NaN
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.068588,124486200,NaN
250,2020-12-29,138.050003,138.789993,134.339996,134.869995,133.270172,121047300,NaN
251,2020-12-30,135.580002,135.990005,133.399994,133.720001,132.133804,96452100,NaN


In [44]:
high_5 = list(data['High'].rolling(5).max())[5:]
max_high_lag = list(data['High'].rolling(5).apply(lambda x: list(x).index(max(x))+1))[5:]

empty_ = [0]*5

high_5 = high_5 + empty_
max_high_lag = max_high_lag + empty_

data['high_5'] = high_5
data['max_high_lag'] = max_high_lag


In [46]:
def add_rolling_columns(column_name, rows_forward):
    rolling_max = list(data[column_name].rolling(rows_forward).max())[rows_forward:]
    max_lag = list(data[column_name].rolling(rows_forward).apply(lambda x: list(x).index(max(x))+1))[rows_forward:]
    empty_ = [0]*rows_forward
    rolling_max = rolling_max + empty_
    max_lag = max_lag + empty_
    data['{}_{}'.format(column_name, rows_forward)] = rolling_max
    data['{}_lag_{}'.format(column_name, rows_forward)] = max_lag
    return data

In [48]:
add_rolling_columns('High', 5).head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,High_max,high_5,max_high_lag,High_5,High_lag_5
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.561546,135480400,77.607498,77.607498,5.0,77.607498,5.0
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.846359,146322800,78.167503,78.167503,5.0,78.167503,5.0
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,73.426834,118387200,79.267502,79.267502,5.0,79.267502,5.0
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,73.081490,108872000,79.392502,79.392502,5.0,79.392502,5.0
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,74.257103,132079200,79.392502,79.392502,4.0,79.392502,4.0
5,2020-01-09,76.809998,77.607498,76.550003,77.407501,75.834381,170108400,79.392502,79.392502,3.0,79.392502,3.0
6,2020-01-10,77.650002,78.167503,77.062500,77.582497,76.005821,140644800,79.684998,79.684998,5.0,79.684998,5.0
7,2020-01-13,77.910004,79.267502,77.787498,79.239998,77.629646,121532000,79.754997,79.754997,5.0,79.754997,5.0
8,2020-01-14,79.175003,79.392502,78.042503,78.169998,76.581383,161954400,79.997498,79.997498,5.0,79.997498,5.0
9,2020-01-15,77.962502,78.875000,77.387497,77.834999,76.253197,121923600,79.997498,79.997498,4.0,79.997498,4.0


In [11]:
max_val = data['High'].shift(5).max()


137.97999572753906

In [10]:
data['High'].shift(-10).max()

138.7899932861328

In [34]:
import numpy as np
def forward_calc(df, column, n, calc_type):

    # check if valid calc type
    if calc_type not in ['min', 'max']:
        raise ValueError('calc_type must be one of min or max')

    # get length of df
    df_len = len(df)
    
    # set up list to store results
    results = []
    
    # loop through each row
    for i in range(df_len):
        # get the index of the nth row
        row_index = i + n
        
        # if nth row is not in df
        if row_index >= df_len:
            # store None in results
            results.append(None)
        else:
            # get column values from nth row
            col_vals = df.loc[i:row_index][column]
            
            # use appropriate calculation
            if calc_type == 'min':
                res = np.min(col_vals)
            else:
                res = np.max(col_vals)
            # store result in results
            results.append(res)
            
    var_ = column + "_" + calc_type
    df[var_] = results
    
    # return list of results
    return df

In [35]:

forward_calc(data,'High',5,'max').tail(12)

,Date,Open,High,Low,Close,Adj Close,Volume,target,High_max
240,2020-12-14,122.599998,123.349998,121.540001,121.779999,120.335449,79184500,1,129.580002
241,2020-12-15,124.339996,127.900002,124.129997,127.879997,126.363091,157243700,1,134.410004
242,2020-12-16,127.410004,128.369995,126.559998,127.809998,126.293922,98208600,0,134.410004
243,2020-12-17,128.899994,129.580002,128.039993,128.699997,127.173363,94359800,0,134.410004
244,2020-12-18,128.960007,129.100006,126.120003,126.660004,125.157578,192541500,1,137.339996
245,2020-12-21,125.019997,128.309998,123.449997,128.229996,126.708931,121251600,1,138.789993
246,2020-12-22,131.610001,134.410004,129.649994,131.880005,130.315643,168904800,0,138.789993
247,2020-12-23,132.160004,132.429993,130.779999,130.960007,129.406555,88223700,0,NaN
248,2020-12-24,131.320007,133.460007,131.100006,131.970001,130.404572,54930100,0,NaN
249,2020-12-28,133.990005,137.339996,133.509995,136.690002,135.068588,124486200,0,NaN


In [26]:
data.reset_index(inplace=True)